In [ ]:
TODO: make it work for norm computation. Now have a technical TF error (precondition error)

In [1]:
"""
This tutorial shows how to generate adversarial examples using FGSM
and train a model using adversarial training with TensorFlow.
It is very similar to mnist_tutorial_keras_tf.py, which does the same
thing but with a dependence on keras.
The original paper can be found at:
https://arxiv.org/abs/1412.6572
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from model import *
import functools
from cleverhans import initializers

from tqdm import tqdm
import logging
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import flags

from cleverhans.loss import CrossEntropy
from cleverhans.dataset import MNIST
from cleverhans.utils_tf import model_eval
from cleverhans.train import train
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils import AccuracyReport, set_log_level, get_log_level
from cleverhans_tutorials.tutorial_models import ModelBasicCNN
from cleverhans.model import Model

%matplotlib inline
from matplotlib import pyplot as plt

Using TensorFlow backend.


Initialized TensorFlow


In [2]:
FLAGS = flags.FLAGS

NB_EPOCHS = 6
BATCH_SIZE = 128
LEARNING_RATE = 0.001
CLEAN_TRAIN = True
BACKPROP_THROUGH_ATTACK = False
NB_FILTERS = 64

In [109]:
class ModelCNN(Model):
  def __init__(self, scope, nb_classes, nb_filters, **kwargs):
    del kwargs
    Model.__init__(self, scope, nb_classes, locals())
    self.nb_filters = nb_filters

    # Do a dummy run of fprop to make sure the variables are created from
    # the start
    self.fprop(tf.placeholder(tf.float32, [128, 28, 28, 1]))
    # Put a reference to the params in self so that the params get pickled
    self.params = self.get_params()

  def eigens(self, sess, xs):
    with sess.as_default():
        x = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.fprop(x)
        return sess.run(self.norms, feed_dict = {x: xs})
    
  def fprop(self, x, **kwargs):
    del kwargs
    my_conv = functools.partial(
        tf.layers.conv2d, activation = None,
        kernel_initializer=initializers.HeReLuNormalInitializer, use_bias = False)
    def get_norm(y):
        return tf.norm(tf.reshape(y, (-1, np.prod([int(t) for t in y.shape[1:]]))), axis = 1)
    with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
      norms = []
      y = x
      norms += [get_norm(y)]
      y = my_conv(y, self.nb_filters, 8, strides=2, padding='same')
      norms += [get_norm(y)]
    
      y = tf.nn.relu(y)
      norms += [get_norm(y)]
      y = my_conv(y, 2 * self.nb_filters, 6, strides=2, padding='valid')
      norms += [get_norm(y)]
    
      y = tf.nn.relu(y)
      norms += [get_norm(y)]
      y = my_conv(y, 2 * self.nb_filters, 5, strides=1, padding='valid')
      norms += [get_norm(y)]
    
      y = tf.nn.relu(y)
      logits = tf.layers.dense(
          tf.layers.flatten(y), self.nb_classes,
          kernel_initializer=initializers.HeReLuNormalInitializer)
      self.norms = norms
      print('Saved norms tensor')
      return {self.O_LOGITS: logits,
              self.O_PROBS: tf.nn.softmax(logits=logits)}

In [110]:
tf.reset_default_graph()
#with tf.Graph().as_default():
if True:
  config = tf.ConfigProto()
  config.gpu_options.per_process_gpu_memory_fraction = 0.5
  config.gpu_options.allow_growth = True
  sess = tf.Session(config=config)

In [111]:
m = ModelCNN('model3', 10, 10)

Saved norms tensor


In [112]:
# Get MNIST data
mnist = MNIST()
x_train, y_train = mnist.get_set('train')

In [113]:
e = m.eigens(sess, x_train[:10])

Saved norms tensor


FailedPreconditionError: Attempting to use uninitialized value model3/conv2d/kernel
	 [[{{node model3/conv2d/kernel/read}} = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](model3/conv2d/kernel)]]
	 [[{{node model3_1/norm_3/Squeeze/_9}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_90_model3_1/norm_3/Squeeze", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'model3/conv2d/kernel/read', defined at:
  File "/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/site-packages/ipykernel_la

In [75]:
e

NameError: name 'e' is not defined

In [51]:
def mnist_tutorial(train_start=0, train_end=60000, test_start=0,
                   test_end=10000, nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                   learning_rate=LEARNING_RATE,
                   clean_train=CLEAN_TRAIN,
                   testing=False,
                   backprop_through_attack=BACKPROP_THROUGH_ATTACK,
                   nb_filters=NB_FILTERS, num_threads=None,
                   label_smoothing=0.1):
  """
  MNIST cleverhans tutorial
  :param train_start: index of first training set example
  :param train_end: index of last training set example
  :param test_start: index of first test set example
  :param test_end: index of last test set example
  :param nb_epochs: number of epochs to train model
  :param batch_size: size of training batches
  :param learning_rate: learning rate for training
  :param clean_train: perform normal training on clean examples only
                      before performing adversarial training.
  :param testing: if true, complete an AccuracyReport for unit tests
                  to verify that performance is adequate
  :param backprop_through_attack: If True, backprop through adversarial
                                  example construction process during
                                  adversarial training.
  :param label_smoothing: float, amount of label smoothing for cross entropy
  :return: an AccuracyReport object
  """

  tf.reset_default_graph()

  # Object used to keep track of (and return) key accuracies
  report = AccuracyReport()

  # Set logging level to see debug information
  set_log_level(logging.WARNING)

  # Create TF session
  if num_threads:
    config_args = dict(intra_op_parallelism_threads=1)
  else:
    config_args = {}
  config = tf.ConfigProto(**config_args)
  config.gpu_options.per_process_gpu_memory_fraction = 0.5
  config.gpu_options.allow_growth = True
  sess = tf.Session(config=config)

  # Get MNIST data
  mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)
  x_train, y_train = mnist.get_set('train')
  x_test, y_test = mnist.get_set('test')

  #print(np.min(x_train), np.max(x_train)) 0.0 1.0

  # Use Image Parameters
  img_rows, img_cols, nchannels = x_train.shape[1:4]
  nb_classes = y_train.shape[1]

  # Define input TF placeholder
  x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                        nchannels))
  y = tf.placeholder(tf.float32, shape=(None, nb_classes))

  # Train an MNIST model
  train_params = {
      'nb_epochs': nb_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate
  }
  eval_params = {
      'batch_size': batch_size
  }

  # making attack single-coordinate
  fgsm_params = {
      'eps': 0.2,
      'clip_min': 0.,
      'clip_max': 1.
  }
  rng = np.random.RandomState([2017, 8, 30])

  def do_eval(preds, x_set, y_set, report_key, is_adv=None):
    acc = model_eval(sess, x, y, preds, x_set, y_set, args=eval_params, feed = {})
    setattr(report, report_key, acc)
    if is_adv is None:
      report_text = None
    elif is_adv:
      report_text = 'adversarial'
    else:
      report_text = 'legitimate'
    if report_text:
      print('Test accuracy on %s examples: %0.4f' % (report_text, acc))

  if clean_train:
    model = ModelCNN('model1', nb_classes, nb_filters)
    preds = model.get_logits(x)
    loss = CrossEntropy(model, smoothing=label_smoothing)

    def evaluate():
      do_eval(preds, x_test, y_test, 'clean_train_clean_eval', False)

    train(sess, loss, x_train, y_train, evaluate=evaluate,
          args=train_params, rng=rng, var_list=model.get_params(),
          feed = {})

    # Calculate training error
    if testing:
      do_eval(preds, x_train, y_train, 'train_clean_train_clean_eval')

    # Initialize the Fast Gradient Sign Method (FGSM) attack object and
    # graph
    fgsm = FastGradientMethod(model, sess=sess)
    adv_x = fgsm.generate(x, **fgsm_params)
    preds_adv = model.get_logits(adv_x)

    # Evaluate the accuracy of the MNIST model on adversarial examples
    do_eval(preds_adv, x_test, y_test, 'clean_train_adv_eval', True)

    # Calculate training error
    if testing:
      do_eval(preds_adv, x_train, y_train, 'train_clean_train_adv_eval')

  return report

In [52]:
mnist_tutorial()

Saved norms tensor
Saved norms tensor
Saved norms tensor
Saved norms tensor
num_devices:  2
Test accuracy on legitimate examples: 0.9880
Test accuracy on legitimate examples: 0.9909
Test accuracy on legitimate examples: 0.9924
Test accuracy on legitimate examples: 0.9925
Test accuracy on legitimate examples: 0.9929
Test accuracy on legitimate examples: 0.9931
Saved norms tensor
Saved norms tensor
Saved norms tensor
Test accuracy on adversarial examples: 0.4848


In [54]:
sess.run

NameError: name 'sess' is not defined